In [34]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from datasketch import MinHash, MinHashLSH

from time import time # needed to measure the elapsed time
from itertools import zip_longest
from collections import Counter

In [35]:
print("This is the answer to part 1 of the exercise\n")

train_ds_comma=pd.read_csv("train.csv")
# train_ds_line=pd.read_csv("train.csv",sep='|')
test_ds_comma=pd.read_csv("test_without_labels.csv")
# test_ds_line=pd.read_csv("test_without_labels.csv",sep='|')


This is the answer to part 1 of the exercise



In [36]:

total_rows_train=len(train_ds_comma)
part_of_set_train=(int) (total_rows_train*0.1)
x_train=train_ds_comma[0:part_of_set_train]["Content"]
y_train=train_ds_comma[0:part_of_set_train]["Label"]
# print(part_of_set_train) # ten percent of the data  set
# print(x_train)

total_rows_test=len(test_ds_comma)
part_of_set_test=(int) (total_rows_test*0.1)
x_test=test_ds_comma[0:part_of_set_test]["Content"]
# print(part_of_set_test)
# print(x_test)

vctrz=CountVectorizer()
x_train=vctrz.fit_transform(x_train)
x_test=vctrz.transform(x_test)

classifiers={'Random Forest': RandomForestClassifier(),
             'SVM': SVC()}
results={}

for name,clfr in classifiers.items():
    scores=cross_val_score(clfr,x_train,y_train,cv=5,scoring='accuracy')
    results[name]=scores.mean()

print("5-Fold Cross-Validation Results:", results)



5-Fold Cross-Validation Results: {'Random Forest': np.float64(0.8800423815298171), 'SVM': np.float64(0.8783431583244287)}


In [41]:
sorted_results=dict(sorted(results.items(), key=lambda item: item[1],reverse=True))
print(sorted_results)

best_fit_name= list(sorted_results.keys())[0]

best_fit_method=classifiers[best_fit_name]
best_fit_method.fit(x_train,y_train)


test_ds_comma[0:part_of_set_test]['Predicted'] = best_fit_method.predict(x_test)
# test_ds_comma[['Id', 'Predicted']].to_csv('testSet_categories.csv', index=False)

{'Random Forest': np.float64(0.8800423815298171), 'SVM': np.float64(0.8783431583244287)}


C:\Users\User\AppData\Local\Temp\ipykernel_5060\778778343.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_ds_comma[0:part_of_set_test]['Predicted'] = best_fit_method.predict(x_test)
C:\Users\User\AppData\Local\Temp\ipykernel_5060\7